In [4]:
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [5]:
items = pd.read_csv("items.csv")
holiday_events = pd.read_csv("holidays_events.csv")
stores = pd.read_csv("stores.csv")
oil = pd.read_csv("oil.csv")
transactions = pd.read_csv("transactions.csv",parse_dates=['date'])

## train is too be, which has 1.254970*10^8 rows, during coding, we only load 5,000,000 rows
train = pd.read_csv("train.csv")

In [3]:
## add family to each row which describes item_nbr
train['family']= train['item_nbr'].map(items.set_index('item_nbr')['family'])

In [4]:
## train is too be, which has 1.254970*10^8 rows, during coding, we only load 5,000,000 rows
train50 = pd.read_csv("train.csv", nrows=16500000, parse_dates=['date'])
train50['family']= train50['item_nbr'].map(items.set_index('item_nbr')['family'])
train50['date'] = train50['date'].astype('datetime64[ns]')

train50

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family
0,0,2013-01-01,25,103665,7.0,NaN,BREAD/BAKERY
1,1,2013-01-01,25,105574,1.0,NaN,GROCERY I
2,2,2013-01-01,25,105575,2.0,NaN,GROCERY I
3,3,2013-01-01,25,108079,1.0,NaN,GROCERY I
4,4,2013-01-01,25,108701,1.0,NaN,DELI
...,...,...,...,...,...,...,...
16499995,16499995,2014-01-04,36,1081359,5.0,NaN,GROCERY I
16499996,16499996,2014-01-04,36,1082042,4.0,NaN,GROCERY I
16499997,16499997,2014-01-04,36,1083152,1.0,NaN,DAIRY
16499998,16499998,2014-01-04,36,1083196,2.0,NaN,DAIRY


In [5]:
train51 = train50.iloc[:5500000,:]
train52 = train50.iloc[5500001:11000000,:]
train53 = train50.iloc[11000001:,:]
train53

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,family
11000001,11000001,2013-09-11,43,872306,3.0,NaN,GROCERY I
11000002,11000002,2013-09-11,43,872307,3.0,NaN,GROCERY I
11000003,11000003,2013-09-11,43,872308,3.0,NaN,GROCERY I
11000004,11000004,2013-09-11,43,872309,1.0,NaN,GROCERY I
11000005,11000005,2013-09-11,43,873687,2.0,NaN,PERSONAL CARE
...,...,...,...,...,...,...,...
16499995,16499995,2014-01-04,36,1081359,5.0,NaN,GROCERY I
16499996,16499996,2014-01-04,36,1082042,4.0,NaN,GROCERY I
16499997,16499997,2014-01-04,36,1083152,1.0,NaN,DAIRY
16499998,16499998,2014-01-04,36,1083196,2.0,NaN,DAIRY


In [9]:
#####
#find all the national holiday
national = holiday_events.loc[holiday_events['locale']=='National'].loc[holiday_events['type']=='Holiday']

mask = (national['date'] > '2012-12-30') & (national['date'] <= '2013-12-30')
national = national.loc[mask]

national['date'] = national['date'].astype('datetime64[ns]')
national["date+"] = national["date"] + timedelta(days=7)
national["date-"] = national["date"] - timedelta(days=7)

national.iloc[1:-1,:]

,date,type,locale,locale_name,description,transferred,date+,date-
44,2013-02-11,Holiday,National,Ecuador,Carnaval,False,2013-02-18,2013-02-04
45,2013-02-12,Holiday,National,Ecuador,Carnaval,False,2013-02-19,2013-02-05
51,2013-04-29,Holiday,National,Ecuador,Viernes Santo,False,2013-05-06,2013-04-22
52,2013-05-01,Holiday,National,Ecuador,Dia del Trabajo,False,2013-05-08,2013-04-24
56,2013-05-24,Holiday,National,Ecuador,Batalla de Pichincha,False,2013-05-31,2013-05-17
67,2013-08-10,Holiday,National,Ecuador,Primer Grito de Independencia,False,2013-08-17,2013-08-03
72,2013-10-09,Holiday,National,Ecuador,Independencia de Guayaquil,True,2013-10-16,2013-10-02
74,2013-11-02,Holiday,National,Ecuador,Dia de Difuntos,False,2013-11-09,2013-10-26
75,2013-11-03,Holiday,National,Ecuador,Independencia de Cuenca,False,2013-11-10,2013-10-27


In [10]:
holiday_events

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...,...
345,2017-12-22,Additional,National,Ecuador,Navidad-3,False
346,2017-12-23,Additional,National,Ecuador,Navidad-2,False
347,2017-12-24,Additional,National,Ecuador,Navidad-1,False
348,2017-12-25,Holiday,National,Ecuador,Navidad,False


In [7]:
##create a list to collect family names
dfitem = pd.DataFrame(items['family'].unique())
dfitem = dfitem.rename(columns = {0:'family'})

dfitem.head()

,family
0,GROCERY I
1,CLEANING
2,BREAD/BAKERY
3,DELI
4,POULTRY


In [8]:
def trainbyset(trainset, start, end):
    for a in national.date:
        if a < start:
            continue
        if a > end:
            break
            
        aplus = a + timedelta(days=7)
        aminus = a - timedelta(days=7)

        bcur = trainset.loc[(trainset['date'] == a)].groupby(['family']).size().reset_index()
        bpluc = trainset.loc[(trainset['date'] == aplus)].groupby(['family']).size().reset_index()
        bminus = trainset.loc[(trainset['date'] == aminus)].groupby(['family']).size().reset_index()

        bcur[0] = bcur[0]*2
        bcur[0] = bcur[0]-bpluc[0]-bminus[0]

        bcur[0].loc[bcur[0] > 0] = 1
        bcur[0].loc[bcur[0] <= 0] = 0
        
        dfitem[a] = dfitem['family'].map(bcur.set_index('family')[0])
        

In [9]:
#add first 1/3
end1 = national.iloc[4,0] + timedelta(days=1)
start1 = national.iloc[1,0] - timedelta(days=1)
trainbyset(train51, start1,end1)

# second 2/3
end1 = national.iloc[6,0] + timedelta(days=1)
start1 = national.iloc[5,0] - timedelta(days=1)
trainbyset(train52, start1,end1)

# last part
end1 = national.iloc[9,0] + timedelta(days=1)
start1 = national.iloc[7,0] - timedelta(days=1)
trainbyset(train53, start1,end1)

# change all nan into 0
a = dfitem.iloc[30][1]
dfitem = dfitem.replace(a, 0)
dfitem.head()

c:\users\zmtyo\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,family,2013-02-11 00:00:00,2013-02-12 00:00:00,2013-04-29 00:00:00,2013-05-01 00:00:00,2013-05-24 00:00:00,2013-08-10 00:00:00,2013-10-09 00:00:00,2013-11-02 00:00:00,2013-11-03 00:00:00
0,GROCERY I,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
1,CLEANING,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0
2,BREAD/BAKERY,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
3,DELI,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
4,POULTRY,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0


In [10]:
## change the data into the form ready for association test
asstest = dfitem.T
new_header = asstest.iloc[0] #grab the first row for the header
asstest = asstest[1:] #take the data less the header row
asstest.columns = new_header

asstest = asstest.astype(int)
asstest

family,GROCERY I,CLEANING,BREAD/BAKERY,DELI,POULTRY,EGGS,PERSONAL CARE,LINGERIE,BEVERAGES,AUTOMOTIVE,...,HOME AND KITCHEN I,MAGAZINES,HOME CARE,PET SUPPLIES,BABY CARE,SCHOOL AND OFFICE SUPPLIES,PLAYERS AND ELECTRONICS,CELEBRATION,LADIESWEAR,BOOKS
2013-02-11 00:00:00,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2013-02-12 00:00:00,0,0,1,0,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2013-04-29 00:00:00,0,0,1,0,0,1,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2013-05-01 00:00:00,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2013-05-24 00:00:00,1,1,1,1,0,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2013-08-10 00:00:00,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-10-09 00:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-11-02 00:00:00,1,1,0,1,0,1,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2013-11-03 00:00:00,1,1,0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
##create a list to collect family names
asstesttemp = pd.DataFrame(national['description'])
asstesttemp = asstesttemp.iloc[1:10]
asstesttemp

,description
44,Carnaval
45,Carnaval
51,Viernes Santo
52,Dia del Trabajo
56,Batalla de Pichincha
67,Primer Grito de Independencia
72,Independencia de Guayaquil
74,Dia de Difuntos
75,Independencia de Cuenca


In [12]:
##create a list to collect family names
a = range(9)
asstest.insert(0, "Holiday", a) 
asstest.Holiday = asstest.Holiday.astype(str)
asstest.head()

family,Holiday,GROCERY I,CLEANING,BREAD/BAKERY,DELI,POULTRY,EGGS,PERSONAL CARE,LINGERIE,BEVERAGES,...,HOME AND KITCHEN I,MAGAZINES,HOME CARE,PET SUPPLIES,BABY CARE,SCHOOL AND OFFICE SUPPLIES,PLAYERS AND ELECTRONICS,CELEBRATION,LADIESWEAR,BOOKS
2013-02-11 00:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-02-12 00:00:00,1,0,0,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2013-04-29 00:00:00,2,0,0,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2013-05-01 00:00:00,3,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2013-05-24 00:00:00,4,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


In [32]:
atest = asstest
assDaily

family,Holiday,GROCERY I,CLEANING,BREAD/BAKERY,DELI,POULTRY,EGGS,PERSONAL CARE,LINGERIE,BEVERAGES,...,HOME AND KITCHEN I,MAGAZINES,HOME CARE,PET SUPPLIES,BABY CARE,SCHOOL AND OFFICE SUPPLIES,PLAYERS AND ELECTRONICS,CELEBRATION,LADIESWEAR,BOOKS
2013-02-11 00:00:00,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-02-12 00:00:00,1,0,0,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2013-04-29 00:00:00,2,0,0,1,0,0,1,1,0,1,...,0,0,0,0,0,0,0,0,0,0
2013-05-01 00:00:00,3,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2013-05-24 00:00:00,4,1,1,1,1,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2013-08-10 00:00:00,5,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-10-09 00:00:00,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2013-11-02 00:00:00,7,1,1,0,1,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2013-11-03 00:00:00,8,1,1,0,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0


In [25]:
atest.columns

Index(['Holiday', 'GROCERY I', 'CLEANING', 'BREAD/BAKERY', 'DELI', 'POULTRY',
       'EGGS', 'PERSONAL CARE', 'LINGERIE', 'BEVERAGES', 'AUTOMOTIVE', 'DAIRY',
       'GROCERY II', 'MEATS', 'FROZEN FOODS', 'HOME APPLIANCES', 'SEAFOOD',
       'PREPARED FOODS', 'LIQUOR,WINE,BEER', 'BEAUTY', 'HARDWARE',
       'LAWN AND GARDEN', 'PRODUCE', 'HOME AND KITCHEN II',
       'HOME AND KITCHEN I', 'MAGAZINES', 'HOME CARE', 'PET SUPPLIES',
       'BABY CARE', 'SCHOOL AND OFFICE SUPPLIES', 'PLAYERS AND ELECTRONICS',
       'CELEBRATION', 'LADIESWEAR', 'BOOKS'],
      dtype='object', name='family')

In [39]:
assDaily = asstest
assDaily = atest.drop(columns=['GROCERY I','BREAD/BAKERY','DELI','PERSONAL CARE', 'BEVERAGES','AUTOMOTIVE','GROCERY II','HOME APPLIANCES',
                               'PREPARED FOODS', 'BEAUTY','HARDWARE','LAWN AND GARDEN', 'PRODUCE', 'HOME AND KITCHEN II','HOME AND KITCHEN I',
                             'MAGAZINES', 'HOME CARE', 'PET SUPPLIES', 'BABY CARE', 'SCHOOL AND OFFICE SUPPLIES', 
                               'PLAYERS AND ELECTRONICS', 'CELEBRATION'])
assDaily

family,Holiday,CLEANING,POULTRY,EGGS,LINGERIE,DAIRY,MEATS,FROZEN FOODS,SEAFOOD,"LIQUOR,WINE,BEER",LADIESWEAR,BOOKS
2013-02-11 00:00:00,0,0,0,0,0,0,1,0,1,1,0,0
2013-02-12 00:00:00,1,0,0,1,0,0,1,1,0,0,0,0
2013-04-29 00:00:00,2,0,0,1,0,0,1,0,0,0,0,0
2013-05-01 00:00:00,3,1,1,1,1,1,1,1,1,1,0,0
2013-05-24 00:00:00,4,1,0,1,1,1,1,1,1,1,0,0
2013-08-10 00:00:00,5,0,1,0,0,0,0,0,0,0,0,0
2013-10-09 00:00:00,6,0,0,0,0,0,0,0,0,1,0,0
2013-11-02 00:00:00,7,1,0,1,1,0,1,1,1,1,0,0
2013-11-03 00:00:00,8,1,0,1,1,0,1,1,1,0,0,0


In [40]:
frequent_itemsets = apriori(assDaily.iloc[:,1:], min_support=0.2, use_colnames=True)
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x: len(x))
## get rid of the single item and only left the support value over 0.1 sets
#frequent_itemsets = frequent_itemsets[(frequent_itemsets['length'] >= 2)].sort_values(by='support', ascending=False)
frequent_itemsets.head()

,support,itemsets,length
0,0.444444,(CLEANING),1
1,0.222222,(POULTRY),1
2,0.666667,(EGGS),1
3,0.444444,(LINGERIE),1
4,0.222222,(DAIRY),1


In [41]:
rule = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
#rule[rule['lift'] != 1].sort_values(by='confidence', ascending=False)

In [43]:
rule[rule['lift'] >= 1].sort_values(by='lift', ascending=False)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
5489,"(MEATS, LIQUOR,WINE,BEER, CLEANING)","(DAIRY, EGGS, SEAFOOD, FROZEN FOODS)",0.333333,0.222222,0.222222,0.666667,3.000000,0.148148,2.333333
3230,(DAIRY),"(MEATS, LINGERIE, LIQUOR,WINE,BEER, CLEANING, ...",0.222222,0.333333,0.222222,1.000000,3.000000,0.148148,inf
4313,"(FROZEN FOODS, DAIRY, MEATS)","(SEAFOOD, LIQUOR,WINE,BEER, CLEANING)",0.222222,0.333333,0.222222,1.000000,3.000000,0.148148,inf
5742,"(EGGS, DAIRY, MEATS)","(FROZEN FOODS, SEAFOOD, LINGERIE, LIQUOR,WINE,...",0.222222,0.333333,0.222222,1.000000,3.000000,0.148148,inf
4314,"(FROZEN FOODS, MEATS, LIQUOR,WINE,BEER)","(SEAFOOD, DAIRY, CLEANING)",0.333333,0.222222,0.222222,0.666667,3.000000,0.148148,2.333333
...,...,...,...,...,...,...,...,...,...
51,"(LIQUOR,WINE,BEER)",(FROZEN FOODS),0.555556,0.555556,0.333333,0.600000,1.080000,0.024691,1.111111
375,"(LIQUOR,WINE,BEER)","(FROZEN FOODS, MEATS)",0.555556,0.555556,0.333333,0.600000,1.080000,0.024691,1.111111
370,"(FROZEN FOODS, MEATS)","(LIQUOR,WINE,BEER)",0.555556,0.555556,0.333333,0.600000,1.080000,0.024691,1.111111
46,(MEATS),"(LIQUOR,WINE,BEER)",0.777778,0.555556,0.444444,0.571429,1.028571,0.012346,1.037037
